In [1]:
import tkinter as tk
import random
from tkinter import messagebox

# VARIABLES GLOBALES
cartas = []
botones = []
primer_carta = None
segunda_carta = None
intentos = 0
aciertos = 0
tiempo = 60
timer_activo = True
niveles = {"Fácil": 6, "Normal": 8, "Difícil": 10}
nivel_actual = "Fácil"
pairs = niveles[nivel_actual]

# FUNCIONES 
def mostrar_pantalla_inicio():
    pantalla_inicio.pack(fill="both", expand=True)
    frame_juego.pack_forget()


def seleccionar_dificultad(nivel):
    global nivel_actual, pairs
    nivel_actual = nivel
    pairs = niveles[nivel]
    iniciar_juego()
    pantalla_inicio.pack_forget()
    frame_juego.pack(fill="both", expand=True)


def iniciar_juego():
    global cartas, botones, primer_carta, segunda_carta, intentos, aciertos, tiempo, timer_activo, pairs

    primer_carta = None
    segunda_carta = None
    intentos = 0
    aciertos = 0
    tiempo = 60
    timer_activo = True

    cartas = list(range(1, pairs + 1)) * 2
    random.shuffle(cartas)

    for widget in frame_cartas.winfo_children():
        widget.destroy()

    botones.clear()

    columnas = 4
    for i in range(len(cartas)):
        boton = tk.Button(frame_cartas, text="?", width=8, height=4,
                          command=lambda i=i: voltear_carta(i), bg="#6EC5E9", font=("Arial", 14, "bold"))
        boton.grid(row=i // columnas, column=i % columnas, padx=5, pady=5)
        botones.append(boton)

    actualizar_tiempo()
    actualizar_contadores()


def voltear_carta(indice):
    global primer_carta, segunda_carta

    boton = botones[indice]

    if boton['text'] != "?" or segunda_carta is not None:
        return

    boton['text'] = cartas[indice]

    if primer_carta is None:
        primer_carta = indice
    else:
        segunda_carta = indice
        ventana.after(600, comprobar_cartas)


def comprobar_cartas():
    global primer_carta, segunda_carta, intentos, aciertos, pairs

    if cartas[primer_carta] == cartas[segunda_carta]:
        botones[primer_carta]['state'] = 'disabled'
        botones[segunda_carta]['state'] = 'disabled'
        aciertos += 1
    else:
        botones[primer_carta]['text'] = "?"
        botones[segunda_carta]['text'] = "?"

    intentos += 1
    actualizar_contadores()

    primer_carta = None
    segunda_carta = None

    if aciertos == pairs:
        messagebox.showinfo("¡Ganaste!", f"Nivel completado: {nivel_actual}\nIntentos: {intentos}")
        mostrar_pantalla_inicio()


def actualizar_contadores():
    label_intentos.config(text=f"Intentos: {intentos}")
    label_aciertos.config(text=f"Aciertos: {aciertos}/{pairs}")


def actualizar_tiempo():
    global tiempo, timer_activo
    if not timer_activo:
        return

    label_tiempo.config(text=f"Tiempo: {tiempo}s")

    if tiempo <= 0:
        messagebox.showwarning("Tiempo agotado", "¡Se acabó el tiempo!")
        mostrar_pantalla_inicio()
        return

    tiempo -= 1
    ventana.after(1000, actualizar_tiempo)


def reiniciar():
    iniciar_juego()

# INTERFAZ PRINCIPAL
ventana = tk.Tk()
ventana.title("Juego de Memoria")
ventana.geometry("500x550")
ventana.configure(bg="#F0F8FF")

# Pantalla de inicio
pantalla_inicio = tk.Frame(ventana, bg="#F0F8FF")
titulo = tk.Label(pantalla_inicio, text="JUEGO DE MEMORIA", font=("Arial", 28, "bold"), bg="#F0F8FF", fg="#0B5FA4")
titulo.pack(pady=30)

sub = tk.Label(pantalla_inicio, text="Selecciona dificultad", font=("Arial", 16), bg="#F0F8FF")
sub.pack(pady=10)

for nivel in niveles:
    tk.Button(pantalla_inicio, text=nivel, font=("Arial", 14), width=15,
              command=lambda n=nivel: seleccionar_dificultad(n), bg="#1877F2", fg="white").pack(pady=5)

# Pantalla del juego
frame_juego = tk.Frame(ventana, bg="#F0F8FF")
label_tiempo = tk.Label(frame_juego, text="Tiempo: 60s", font=("Arial", 14), bg="#F0F8FF")
label_tiempo.pack(pady=5)

label_intentos = tk.Label(frame_juego, text="Intentos: 0", font=("Arial", 14), bg="#F0F8FF")
label_intentos.pack()

label_aciertos = tk.Label(frame_juego, text="Aciertos: 0", font=("Arial", 14), bg="#F0F8FF")
label_aciertos.pack(pady=5)

frame_cartas = tk.Frame(frame_juego, bg="#F0F8FF")
frame_cartas.pack()

boton_reiniciar = tk.Button(frame_juego, text="Reiniciar Juego", font=("Arial", 12), command=reiniciar, bg="#FFB703")
boton_reiniciar.pack(pady=10)

mostrar_pantalla_inicio()
ventana.mainloop()